# Internship and Junior data analyst finder

To use .ipynb Jupyter notebook, see [How To Set Up Jupyter Notebook with Python 3 on Ubuntu 18.04](https://www.digitalocean.com/community/tutorials/how-to-set-up-jupyter-notebook-with-python-3-on-ubuntu-18-04).


## Goal:

Search for people that attended ML internships or junior positions around Europe. Filter by company size. The objective is to find a list of companies that hire for such positions. I will also leave a count of repetitive positions in the same company, as it helps decide which companies to put on the watchlist.

To get people from linkedin, I will use tom quirk's unofficial [linkedin api](https://github.com/tomquirk/linkedin-api)
I will also use `dotenv` to load the username/password safely.

In [2]:
import re
import os
from diskcache import Cache
from linkedin_api import Linkedin

%load_ext dotenv
%dotenv

LINKEDIN_USERNAME = os.environ.get("LINKEDIN_USERNAME")
LINKEDIN_PASSWORD = os.environ.get("LINKEDIN_PASSWORD")

linkedin = Linkedin(LINKEDIN_USERNAME, LINKEDIN_PASSWORD)

ModuleNotFoundError: No module named 'dotenv'